The ladybug-tools project features different components to integrate BIM projects with energy and climate analysis tools. Most are divided or in the process of being divided) between an independent core component, and GUI components for different BIM apps like Revit or Rhino, mostly using node-based systems like Grasshopper or Dynamo. Everything is written in Python and has a good API.

For the FreeCAD integration, however, I'm thinking of using core funcionality directly, and creating interface panels on the fly, don't know yet. This document is mostly to explore and find interesting functionality.

This document is hosted at https://gist.github.com/yorikvanhavre/0e7ab25d077263a25e1dac0a3c974d28


### Ladybug

[Ladybug](https://github.com/ladybug-tools/ladybug) is a component to interact with weather data. Weather data is typically downloaded from the [energyplus website](https://energyplus.net/weather), maintained by the USA goverment, but has files for most parts of the world. 

Ladybug can be cloned with GIT, but it's easy to install with pip with:

```pip3 install lbt-ladybug```

This is how you work with it (I downloaded one epw file and renamed it "bxl.epw" for simplicity).

#### Contents of ladybug module

In [6]:
import ladybug
[e for e in dir(ladybug) if not e.startswith("_")]

['analysisperiod',
 'compass',
 'datacollection',
 'datatype',
 'designday',
 'dt',
 'epw',
 'futil',
 'header',
 'location',
 'psychrometrics',
 'skymodel',
 'sunpath']

#### Contents of ladybug.epw module:

In [7]:
from ladybug import epw
[e for e in dir(ladybug.epw) if not e.startswith("_")]

['AnalysisPeriod',
 'DesignDay',
 'EPW',
 'EPWField',
 'EPWFields',
 'Header',
 'HourlyContinuousCollection',
 'Location',
 'MonthlyCollection',
 'angle',
 'calc_sky_temperature',
 'distance',
 'division',
 'energyflux',
 'energyintensity',
 'fraction',
 'generic',
 'illuminance',
 'luminance',
 'os',
 'pressure',
 'readmode',
 'speed',
 'temperature',
 'write_to_file',
 'xrange']

In [8]:
epw_data = ladybug.epw.EPW("bxl.epw")
print(epw_data.dry_bulb_temperature)
epw_data

Hourly Continuous Data Collection
1/1 to 12/31 between 0 and 23 @1
Dry Bulb Temperature (C)
...8760 values...


EPW file Data for [BRUSSELS]

#### Contents of epw object:

In [9]:
[e for e in dir(epw_data) if not e.startswith("_")]

['ToString',
 'aerosol_optical_depth',
 'albedo',
 'annual_cooling_design_day_004',
 'annual_cooling_design_day_010',
 'annual_heating_design_day_990',
 'annual_heating_design_day_996',
 'atmospheric_station_pressure',
 'ceiling_height',
 'comments_1',
 'comments_2',
 'convert_to_ip',
 'convert_to_si',
 'cooling_design_condition_dictionary',
 'daylight_savings_end',
 'daylight_savings_start',
 'days_since_last_snowfall',
 'dew_point_temperature',
 'diffuse_horizontal_illuminance',
 'diffuse_horizontal_radiation',
 'direct_normal_illuminance',
 'direct_normal_radiation',
 'dry_bulb_temperature',
 'extraterrestrial_direct_normal_radiation',
 'extraterrestrial_horizontal_radiation',
 'extreme_cold_weeks',
 'extreme_design_condition_dictionary',
 'extreme_hot_weeks',
 'file_path',
 'from_dict',
 'from_missing_values',
 'global_horizontal_illuminance',
 'global_horizontal_radiation',
 'header',
 'heating_design_condition_dictionary',
 'horizontal_infrared_radiation_intensity',
 'import_data

The ladybug project also maintains a site where you can download epw files: https://www.ladybug.tools/epwmap/

You can also create locations or extract location from an epw file:

In [10]:
loc = ladybug.location.Location('Sydney', 'AUS', latitude=-33.87, longitude=151.22, time_zone=10)
loc = epw_data.location
print(loc)

Site:Location,
  BRUSSELS,
  50.9,      !Latitude
  4.53,     !Longitude
  1.0,     !Time Zone
  58.0;       !Elevation


Ladybug can calculate sun paths:

In [11]:
sp = ladybug.sunpath.Sunpath.from_location(loc)
sun = sp.calculate_sun(month=11, day=15, hour=11.0)
print('altitude: {}, azimuth: {}'.format(sun.altitude, sun.azimuth))

altitude: 18.02052748676022, azimuth: 158.46264722289163


Most data in the EPW object are lists of numbers for each hour of the year, like *dry_bulb_temperature* or *relative_humidity* which are useful for energy analyses, but not really directly for the user.

In [12]:
print(epw_data.relative_humidity)

Hourly Continuous Data Collection
1/1 to 12/31 between 0 and 23 @1
Relative Humidity (%)
...8760 values...


In [13]:
365*24

8760

#### Creating a solar diagram

Obtaining directional 3D vectors for 9h, 12h and 15h on March 30th, June 30th, September 30th and December 30th

In [30]:
import math
vectors = []
for month in (3,6,9,12):
    for hour in (9,12,15):
        sun = sp.calculate_sun(month=month, day=30, hour=hour)
        xyangle = 90 + sun.azimuth # use X direction as base
        x = math.cos(math.radians(xyangle))
        y = math.sin(math.radians(xyangle))
        z = math.cos(math.radians(sun.altitude))
        vectors.append((x,y,z))
print(vectors)
    

[(-0.9082125488882267, -0.4185092185865809, 0.9166072437014305), (-0.26937811050451205, -0.9630344924150012, 0.7424663286416248), (0.6741897527703045, -0.7385581745939285, 0.815188618704883), (-0.9820592723540658, -0.18857249423869546, 0.7795769997217051), (-0.3732549295631947, -0.9277288168192117, 0.4867536841253755), (0.8264136162967166, -0.5630635264331931, 0.615548708213317), (-0.8476997769708247, -0.5304762842235402, 0.9402295952215028), (-0.17025295274041632, -0.9854003917612218, 0.8118084839575337), (0.6783988696012553, -0.7346937958930503, 0.8867106229004053), (-0.7633756061356852, -0.6459548621668353, 0.9996954244183549), (-0.18383923938399696, -0.9829563235783741, 0.9642046328113717), (0.5207118254072426, -0.8537325078038538, 0.9839803824126283)]


#### Creating a wind rose
Getting a series of 3D vectors for each portion of a circle, representing a wind itensity (max 1.0)

In [31]:
sectors = 24 # the number of sectors to divide the circle into

baseangle = 360/sectors
sectorangles = [i * baseangle for i in range(sectors)] # the divider angles between each sector
basebissect = baseangle/2
angles = [basebissect] # build a list of central direction for each sector
for i in range(1,sectors):
    angles.append(angles[-1]+baseangle)

windsbysector = [0 for i in range(sectors)] # prepare a holder for values for each sector
for hour in epw_data.wind_direction:
    sector = min(angles, key=lambda x:abs(x-hour)) # find the closest sector angle
    sectorindex = angles.index(sector)
    windsbysector[sectorindex] = windsbysector[sectorindex] + 1
maxwind = max(windsbysector)
windsbysector = [wind/maxwind for wind in windsbysector] # normalize

vectors = [] # create 3D vectors
for i in range(sectors):
    angle = math.radians(90 + angles[i])
    x = math.cos(angle) * windsbysector[i]
    y = math.sin(angle) * windsbysector[i]
    vectors.append((x,y,0))
print(vectors)
    
    

[(-0.027054519841974333, 0.2054994803574807, 0), (-0.047313588001502004, 0.11422510583775909, 0), (-0.04205988054969343, 0.054813503511030794, 0), (-0.18925083317492733, 0.14521727179262578, 0), (-0.13841395177987278, 0.05733293604887892, 0), (-0.16223643186116898, 0.021358831454190257, 0), (-0.06417352193619574, -0.008448604441879686, 0), (-0.16193088533470554, -0.06707396887271753, 0), (-0.03461905484907208, -0.026564135084016904, 0), (-0.07659325615891543, -0.09981827481482448, 0), (-0.050931685907499184, -0.12295996687241127, 0), (-0.04651478850023658, -0.3533148960532124, 0), (0.03540819614405755, -0.2689519514854046, 0), (0.2922309847151596, -0.7055080066449826, 0), (0.21738317210420494, -0.28329926551490653, 0), (0.7933533402912349, -0.6087614290087209, 0), (0.3675360758426719, -0.15223842727542108, 0), (0.5256460392301875, -0.06920261391157649, 0), (0.22424680137254913, 0.029522651476680668, 0), (0.37963049881372873, 0.15724810129910968, 0), (0.08943255836010289, 0.068624015633

## Honeybee

This is the honeybee module. Install with:
```
pip3 install lbt-honeybee
pip3 install honeybee-radiance
pip3 install honeybee-energy
```
The honeybee module basically allows you to build energy analysis (gbxml) models and run through radiance or energyplus


In [7]:
import honeybee
print(dir(honeybee))

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_base', '_basewithshade', '_lockable', 'altnumber', 'aperture', 'boundarycondition', 'config', 'door', 'extensions', 'extensionutil', 'face', 'facetype', 'finder', 'get_logger', 'importlib', 'ispkg', 'logger', 'logutil', 'name', 'orientation', 'pkgutil', 'properties', 'room', 'shade', 'typing', 'writer']


In [19]:
# script below from https://github.com/ladybug-tools/honeybee

from honeybee.room import Room
from honeybee.radiance.material.glass import Glass
from honeybee.radiance.sky.certainIlluminance import CertainIlluminanceLevel
from honeybee.radiance.recipe.pointintime.gridbased import GridBased

# create a test room

room = Room(origin=(0, 0, 3.2), width=4.2, depth=6, height=3.2, rotation_angle=45)
print(room)

# add fenestration

# # add a window to the back wall

room.add_fenestration_surface(wall_name="back", width=2, height=2, sill_height=0.7)

# add another window with custom material. This time to the right wall

glass_60 = Glass.by_single_trans_value("tvis_0.6", 0.6)
room.add_fenestration_surface("right", 4, 1.5, 1.2, radiance_material=glass_60)

# run a grid-based analysis for this room

# generate the sky

sky = CertainIlluminanceLevel(illuminance_value=2000)

# generate grid of test points

analysis_grid = room.generate_test_points(grid_size=0.5, height=0.75)

# put the recipe together

rp = GridBased(sky=sky, analysis_grids=(analysis_grid,), simulation_type=0,
hb_objects=(room,))

# write and run the analysis

batch_file = rp.write(target_folder="/home/yorik/ladybug", project_name="room")
rp.run(batch_file, debug=False)

# results - in this case it will be an analysis grid

result = rp.results()[0]

# print the values for each point

for value in result.combined_value_by_id():
    print("illuminance value: %d lux" % value[0])

ModuleNotFoundError: No module named 'honeybee.radiance'

In [17]:
import honeybee_radiance
print(dir(honeybee_radiance))

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_extend_honeybee', 'config', 'dynamic', 'geometry', 'get_logger', 'honeybee_radiance', 'lib', 'lightpath', 'logger', 'modifier', 'modifierset', 'mutil', 'primitive', 'properties', 'reader', 'sensor', 'sensorgrid', 'writer']


The above contents doesn't correspond to the examples nor what's on github... I suspect there is some big version difference between the version on pip and the one being developed. Not sure which one to take from now...

Also, it seems honeybee.room can only create cubic rooms.